In [1]:
!python -V

Python 3.10.13


In [2]:
import pandas as pd

#import seaborn as sns
#import matplotlib.pyplot as plt

In [3]:
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60 # turn seconds to minutes

    df = df[(df.duration >= 1) & (df.duration <= 60)] # remove very short and very long rides

    categorical = ['PULocationID', 'DOLocationID'] # pick up location and drop off location
    df[categorical] = df[categorical].astype(str) # turn into strings
    
    return df

In [6]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet')

In [7]:
len(df_train), len(df_val)

(59603, 66097)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
from sklearn.pipeline import make_pipeline

In [10]:
pipeline = make_pipeline(
    DictVectorizer(),
    LinearRegression()
)

In [11]:
# Will use only 3 features fro the model
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [12]:
pipeline.fit(train_dicts, y_train)

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('linearregression', LinearRegression())])

In [13]:
y_pred = pipeline.predict(val_dicts)
mean_squared_error(y_val, y_pred, squared=False)

8.19383255249626

In [14]:
#sns.histplot(y_pred, kde=True, stat="density", color='blue', bins=25, label='prediction')
#sns.histplot(y_val, kde=True, stat="density", color='orange', bins=40, label='actual')
#plt.legend();

In [16]:
with open('lin_reg.bin', 'wb') as f_out:
    pickle.dump(pipeline, f_out)